# TME 7 : Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les suffix trees

Nous allons utiliser l'algorithme suffix-tree pour la recherche rapide et éfficace de motifs. Un suffix-tree est construit à partir d'un jeux de séquences, ensuite nous pouvons rechercher le motif en temps O(k) où k est la longueur du motif.

1\. Nous allons réutiliser les fonctions du TME6 pour générer ``t`` séquences artificielles de taille ``n``, et implanter dans chaque séquence un motif de taille ``k`` à des positions aléatoires avec ``v`` substitutions choisies aléatoirement. Cependant, les ``t`` séquences artificielles initiales (sans implantation) ainsi que le motif initial (sans variation/mutation) doivent être générées une seule fois. Ensuite, selon chaque question, nous introduisons des différentes variation au motif initial et les implantons dans les séquences afin de générer des nouveau jeux de données. 

In [141]:
import math
import random
from random import *
import operator
import numpy as np


nuc = ('A', 'C', 'G', 'T')

k=8 #taille de motif
v=0 #nb de positions variables dans le motif
t=5 #nb de sequences
n=100 #longuer des sequence


1.1\. Generer les séquences artificielles initiales et implanter un motif (sans variation, v=0)

In [16]:
def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    if upper:      
        for i in range(0,n) :
            sequence+= choice(nuc).upper()
    else:
        for i in range(0,n) :
            sequence+= choice(nuc).lower()
   
    return sequence


def posrand(taille, nb_nu):
    """
    taille : len du motif
    nb_nu : nb de nucleotides a changer
    choisi dans une sequence les positions où insérer les motifs
    renvoie une liste de int #position
    """
    listpos = []
    i = 0
    while i < nb_nu:
        rand = randint(0,taille-1)
        if rand not in listpos :
            listpos.append(rand)
            i += 1
    return sorted(listpos)


def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = ''
    #je transforme le motif en liste pour changer les nuc
    listemotif = list(motif)
    listePos = posrand(len(motif),nbpos)
    
    for n in listePos :
        listemotif[n] = choice(nuc)
        
    motifM = "".join(listemotif)
    
    if upper :
        motifM = motifM.upper()
    else :
        motifM = motifM.lower()

    return motifM


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
  
    sequences = []
    taille = n-k
    motif = generateRandomSequence(k)
    
    print("Le motif : " + motif)
    for i in range(t):
        seq = generateRandomSequence(taille,True)
        
        listeseq = list(seq)
        listeseq.insert(randint(0,taille-1), modifierMotif(motif, v))
        seq = ''.join(listeseq)
        sequences.append(seq)
    

    return sequences

def implantMotif(sequences, motif, k, v, t, n):
    """
    Génère des séquences aléatoires et implante des motifs (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations (si v==0 implante un motif sans variations)
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNAOrig : matrice de dimension t x n-k avec les sequences aléatoires
    sortie DNAMot: matrice de dimension t x n avec les sequences aléatoires et les motifs implantés
    sortie motifFix: motif sans variations
    REMARQUE : La taille totale des séquences plus motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    Cette fonction permet de soit générer les séquences et le motif, soit juste implanter de motif (avec ou sans variations) à des séquences et motif existantes (passé en paramètre)
    """
    DNAOrig = []
    DNAMot = []
    motif = generateRandomSequence(k)
    
    for i in range(t):
        seq = generateRandomSequence(n-k)
        DNAOrig.append(seq)
        
        listeseq = list(seq)
        listeseq.insert(randint(0,n-k-1), modifierMotif(motif, v))
        seq = ''.join(listeseq)
        DNAMot.append(seq)

    return DNAOrig, DNAMot, motif


adnOri, adn, fix_motif = implantMotif(None, '', k, 0, t, n) #générer les séquences et le motif sans variation
print (adnOri)
print (adn)


adn  = [s.upper() for s in adn]
#print (adn)
print (fix_motif)

['CATGTCCAAGCGTCGTAGTGACCTGTTGATGCACCGCCCTTACTACTATCTAGGCTATAGGTGCGATTGATTCTGTCATGTTGGAAAATTGC', 'CAATGCCCCAAGTCCTCACCTTAACTCTTGAAAATTCAGCACCCCATGATCTATTGTTAAGTATCCAGTGGATCATGCAGCCAAGCCTAGGC', 'ACAGAAGAGGCAAATCCCTACAACGAATTCGACGAAAACTGCAGACTATCGTTTATCTTACAAAATAATGAGGCTTTACGTTGGCACAGCCT', 'GAACAAACGTTGGTATCGGACGTTGCCAACGAGGCCGTAGCGGAGATGGCAGCTTGCTATATGGGGGGCTGTCAGGCTTTTGGACGGAGTCC', 'GTCGGAGAACAGGCCAGCGCAACAGTTTTCAGTCCGTGAGGTATACATATGAAGAGCTCCCTGTGCTGGTGGGAACGCCGTTGCGTGTGCAC']
['CATGTCCAAGCGTCGTAGTGACCTGTTGATGCACCGCCCTTACTACTATCTAGGCTATAGGTGCGATTGATTCTGTCATGTTGGAATTGCCTGAAATTGC', 'CAATGCCCCAAGTCCTCAATTGCCTGCCTTAACTCTTGAAAATTCAGCACCCCATGATCTATTGTTAAGTATCCAGTGGATCATGCAGCCAAGCCTAGGC', 'ACAGAAGAGGCAAATATTGCCTGCCCTACAACGAATTCGACGAAAACTGCAGACTATCGTTTATCTTACAAAATAATGAGGCTTTACGTTGGCACAGCCT', 'GAACAAACGTTGGTATCGGACGTTGCCAACGAGGCCGTAGCGGAGATGGCAGCTTGCTATATGGGGGATTGCCTGGCTGTCAGGCTTTTGGACGGAGTCC', 'GTCGGAGAACAGGCCAGCGCAACAGTTTATTGCCTGTCAGTCCGTGAGGTATACATATGAAGAGCTCCCTGTGCTGGTGGGAACGCCGTTGCGTGTGCAC'

2\. Définissez une fonction ``construct_tree`` pour construire un suffix tree à partir des séquences artificielles (après implantation) en utilisant le python package suffix-trees trouvable ici: https://pypi.org/project/suffix-trees/. Tester si votre fonction est capable de trouver le motif sans variation implanté.

In [17]:
!pip install suffix-trees

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [27]:
from suffix_trees import STree

st = STree.STree("abcdefghab")
print(st.find("abc")) # 0
print(st.find_all("ab")) # [0, 8]

def construct_tree(sequences):
    """
    construire un abre de suffixe
    entrée1 sequences : matrice de dimension txn avec les sequences
    sortie1 st : arbre de suffixe
    REMARK: Vous devez concatener toutes les sequences de la matrice avant d'appeller la fonction STree
    """
    st = STree.STree(sequences)
    return st

tree = construct_tree(adn)
print(tree.find_all(fix_motif))




0
{8, 0}
{432, 370, 85, 119, 217}


3\. Avant de chercher les motifs, implémentez ou reutilisez les fonctions pour générer tous les motifs (k-mer) possibles de taille k, en éliminant les motifs peu complexe pour éviter les calculs inutiles.

In [47]:
from itertools import product

def kmerList(allkmers):
    """
    On transforme le resultat du product en liste
    """
    Liste = []
    for mo in allkmers :
        Liste.append("".join(list(mo)))
        
    return Liste


#Enlever les motifs peu complexe
#quand une lettre est repetée m fois (ou par defaut 5 fois)
#ou quand deux lettres consecutifs sont répétés (ex AGAG)

#Enlever les motifs peu complexe
#quand une lettre est repetée m fois (ou par defaut 5 fois)
#ou quand deux lettres consecutifs sont répétés (ex AGAG)

def removeLowComplexe(kmers, m = 5):
    """
    entree kmers : Liste de tous les K-mers de taille k
    sortie tout les K-mers en enlevant ceux qui sont peu complexe
    """
    resultat = []
    for mo in kmers :
        valid = True
        cpt = 0
        
        for i in range(len(mo)-1) :
            #on regarde si deux lettres consécutifs sont répétées
            if i < len(mo)-3 :
                if mo[i] == mo[i+2] and mo[i+1] == mo[i+3] : 
                    valid = False
                    break
            #on regarde si une lettre est répétée m fois
            if mo[i] == mo[i+1] :
                cpt+= 1
                if cpt >= m-1 :
                    valid = False
                    break
            else :
                cpt = 0
                
        if valid == True :
            resultat.append(mo)
            
    return resultat

allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)


print (len(kmers))
kmersValid = removeLowComplexe(kmers)
print (len(kmersValid))




65536
49680


4\. **Exact matching:** Définissez la fonction ``exact_match`` qui cherche dans le suffix tree tous les motifs possibles (k-mers), générés à la question precedent. La fonction renvoie un dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrence (valeurs). Ce dictionnaire doit être trié par nombre d'occurrences. 

Ensuite, trouvez et affichez les 10 motifs plus fréquents dans notre jeux de données artificiels.

In [67]:
def exact_match(kmersV, stree):
    """
    Cherche dans le suffix tree tous les motifs possibles
    entrée1 kmersV: Liste de Kmers à chercher
    entrée2 stree: suffix tree
    sortie1 motif_occur_sorted: dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrences (values).
    """
    motif_occur = {}
    
    for mer in kmersV :
        tmp = stree.find_all(mer)
        if len(tmp) > 0 : 
            motif_occur[mer] = len(tmp)
    
    motif_occur = {ka: v for ka, v in sorted(motif_occur.items(), key=lambda item: item[1],reverse=True)}
    return motif_occur

st = construct_tree(adn)
motif_occur = exact_match(kmersValid, st)

i = 0
while i < 10 : 
    kee = list(motif_occur.keys())[i]
    print(kee, motif_occur[kee])
    i+=1

ATTGCCTG 5
AATTGCCT 2
TATTGCCT 2
TGCCTGCC 2
TTGCCTGC 2
AAACGTTG 1
AAACTGCA 1
AAATAATG 1
AAATTCAG 1
AACAAACG 1


5\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Faites attention aux valeurs élevées des variable k, t, et n par rapport aux TMEs précedants. Quelle est la complexité de chaque recherche de motif? 

reponse : Oui, le motif ATTGCCTG a été retrouvé, l'algorithme a été très rapide.

6\. Introduisez deux variation (v=2) au motif initial. Pour cela avant de chaque implantation, créez d'abord un motif varié (avec v substitutions choisies aléatoirement) à partir du motif initial et puis implantez-le dans une séquence. Repetez pour chaque sequence dans le Jeux de donnée. Il suffit de mettre ``v`` égal ``2`` et réutiliser les fonctions définies à la question 1.

In [150]:
v=2
adnOri, adn,  fix_motif = implantMotif(adnOri, fix_motif, k, v, t, n)
print (adn)
adn  = [s.upper() for s in adn]
print('Fixed motif: ', fix_motif)


['CCAGAAGGAACCATCCGGTGAGTCATTTTATGTCAGTTAATCGCCGGTCTTCGCACCAGCGAGAAGCATTATTTAAAAACTCAATAAGAGCCTCTCGATA', 'ATCCGAAATGGACACGAATCAGTGTCTCACTGAAGCGGTCTTGGTGGGATAGGTTCGATATACATCGGGAAGCGAATCGGAGCAAAGCCTGATCGATGTG', 'TCAAAGGTCCAAACTCATAGGGATGATTCACTTTGACGCAAAGTACACATACAGAGATTATTGACTGGAAATGGGCGAGGCCGCGAAGCGGCGGCTCCCG', 'TAACTTCGCCCGCGAAACTGTTGCACTCTACGCCTTAGATCACGCGCAGACATTACGATCCCCGTGGTTCGCATTACCGGGGTTCATCGGCGCTCTTACA', 'ATCTCGGGTTTCATAGGCACGCTTGTAGCATTGTAACCCCCGGGGTTTCTATAGCACTCGGCAGTATGTTTAAAGGACTATGTCGGGCTTACAGCAATTG']
Fixed motif:  AATCGGCG


7\. Construisez le suffix tree à nouveau à partir des nouvelles séquences en utilisant le python package suffix-trees.

In [151]:
st = construct_tree(adn)

8\. **Inexact matching:** 

Définissez fonction ``inexact_match`` qui cherche tous les motifs possibles (k-mers) générés à la question 2 dans le nouveau suffix tree donné (construit à partir des nouvelle séquences qui incluent le motif varié), et renvoie un dictionnaire qui contient les motifs (keys) et les listes de toutes leurs variations (values). Il faut que vous utilisiez la technique *seed* pour trouver le motif variable. 

Ensuite afficher les top motifs (les sequence consensus) de meilleurs scores.
Pour calculer les scores utilisér les fonctions precedents. par exemple, le score d'une matrice de fréquence est la somme de max de chaque colonne.



In [152]:
def inexact_match(kmersV, sequences, stree, v):
    """
    cherche de motif variables dans un suffix tree
    entrée1 kmersV: liste de motifs à chercher
    entrée2 sequences: matrice de dimension txn avec les sequences
    entrée3 stree: suffix Tree
    entrée4 v: nombre de variations dans le motifs
    sortie1 motif_match: dictionnaire clé=sequence consensus des motifs; value= list de motifs.
    """
    motif_match = {}
    text = '' 
    for s in sequences : 
        text = text + s + ' ' 
    
    for mer in kmersV : 
        ns = v +1
        ls = int(len(mer)/ns)
        
        for i in range(ns-1) : 
            seed_mer = mer[i*ls:i*ls+ls]
            fa = stree.find_all(seed_mer)
            if len(fa) > 1 : 
                for cp in fa :
                    try : 
                        seed_text = text[cp-i*ls:cp+len(mer)-i*ls]
                        if ' ' in seed_text :
                            continue
                    except IndexError : 
                        continue
                        
                    diff = 0
                    j = 0
                    while j > len(mer) and diff>v :
                        if mer[j] == seed_text[j] :
                            diff += 1
                        j += 1
                            
                    if diff <= v :
                        if mer in motif_match :
                            motif_match[mer].append(seed_text)
                        else :
                            motif_match[mer] = [seed_text]
                            
    return motif_match


#Test
seqTest = "banananabanbnaabanbna"
st1 = construct_tree([seqTest])
motif_match = inexact_match(['banbna'], [seqTest], st1, 1)
print(motif_match)
#{'banbna': ['banana', 'banbna', 'banbna']}




def printTopMotifsScore(motif_match, top):
    """
    Affiche les Top Motifs ayant les meilleurs scores 
    entrée1 motif_match: dictionnaire clé=sequence consensus des motifs; value= list de motifs
    entrée top: la valeur à consider pour l'affichage
    sortie None
    """
    nv_dico = {}
    for ka, val in motif_match.items() :
        nv_dico[ka] = len(val)
    print(nv_dico["GTAACCAG"])
        
    nv_dico = {ka: v for ka, v in sorted(nv_dico.items(), key=lambda item: item[1],reverse=True)}
    i = 0
    while i < top : 
        kee = list(nv_dico.keys())[i]
        print(kee, nv_dico[kee])
        i+=1

top = 10
motif_match1 = inexact_match(kmersValid, adn, st, v)

printTopMotifsScore(motif_match1, top)

#GTAACCAG


{'banbna': ['banana', 'banbna', 'banbna']}
55
GGCGAAAC 72
GGCGAAAG 72
GGCGAAAT 72
GGCGAACA 72
GGCGAACC 72
GGCGAACG 72
GGCGAACT 72
GGCGAAGA 72
GGCGAAGC 72
GGCGAAGG 72


9\. Créez le motif logo à partir des séquences du meilleur motif variable que vouz venez de trouver. Vous pouvez utilizer ce site: https://weblogo.berkeley.edu/logo.cgi. Affichez votre logo ci-dessous.

Motif LOGO:
<img src="logo_1.png">

10\. Avez-vous réussi à trouver votre motif initial implanté en séquences? l'algorithme était-il rapide? Quelle est la complexité de l'algorithme?

11\. Tester l'algorithme  suffix tree sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [ ]:
def readFasta(genome):
    sequence = []
    file = open(genome, "r")
    sequences = []
    seq = ""
    for s in file:
        if s[0] != ">":
            seq += s.strip().upper()
        else:
            sequences.append(seq)
            seq = ""
    return sequences[1:]
genome = "Sequence_by_Peaks_0.fasta"
sequences = readFasta(genome)
#print(sequences)

Motif LOGO:
<img src="logo_2.png">